<a href="https://colab.research.google.com/github/Chia-Yin-Lee/NTHU/blob/main/businesstimes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title show your CPU or GPU details
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6681616048898335689
 xla_global_id: -1]

In [ ]:
#@title connect google drive folder

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/thesis

Mounted at /content/drive
/content/drive/MyDrive/thesis


In [ ]:
#@title import packages

import yfinance as yf
import pandas as pd
import numpy as np

import copy
import datetime as dt
from datetime import datetime
import string
import codecs

import re, collections
from collections import defaultdict, Counter


try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterator, *args, **kwargs):
        return iterator

In [ ]:
#@title SREIT symbols & sectors

# Divide the SREITs with diff sectors
healthcare_symbols = ["AW9U.SI", "C2PU.SI"]
hospitality_symbols = ["HMN.SI", "J85.SI", "Q5T.SI", "ACV.SI"]
retail_symbols = ["BMGU.SI", "C38U.SI", "AU8U.SI", "CEDU.SI", "J69U.SI", "D5IU.SI", "CRPU.SI", "SK6U.SI", "P40U.SI"]
office_symbols = ["UD1U.SI", "K71U.SI", "JYEU.SI", "N2IU.SI", "TS0U.SI", "OXMU.SI", "T82U.SI"]
industrial_symbols = ["O5RU.SI", "A17U.SI", "DHLU.SI", "BWCU.SI", "J91U.SI", "BUOU.SI", "AJBU.SI", "ME8U.SI", "M44U.SI", "M1GU.SI"]
seit_symbols = ["AW9U", "C2PU", "HMN", "J85", "Q5T", "ACV","BMGU", "C38U", "AU8U", "CEDU", "J69U", "D5IU", "CRPU", "SK6U", "P40U",
                "UD1U", "K71U", "JYEU", "N2IU", "TS0U", "OXMU", "T82U", "O5RU", "A17U", "DHLU", "BWCU", "J91U", "BUOU", "AJBU", "ME8U", "M44U", "M1GU"]

# Define the date range for the data
start_date = "2022-04-26"
end_date = "2023-08-31"

In [ ]:
df = pd.read_csv('TheBusinessTimes.csv')

df.head()

,Date,Headline,Link,Contents
0,2022-04-11,Data centre S-Reits committed to sustainable g...,https://www.businesstimes.com.sg/companies-mar...,THE data centre industry has experienced treme...
1,2022-04-11,S-Reits punching above their weight in the glo...,https://www.businesstimes.com.sg/wealth/wealth...,SINGAPORE Real Estate Investment Trusts (Reits...
2,2022-04-14,"I'm not the kingmaker,' says Sabana Reit direc...",https://www.businesstimes.com.sg/companies-mar...,SABANA Industrial Real Estate Investment Trust...
3,2022-04-14,Property listcos have more women chair than th...,https://www.businesstimes.com.sg/property/prop...,CLOSE to 18 per cent of the real estate servic...
4,2022-04-19,Time to double down on S-Reit investments amid...,https://www.businesstimes.com.sg/opinion-featu...,"INFLATION is poised to continue rising, as sig..."


In [ ]:
column_names = ['Date', 'Headline', 'Contents']

new_df = pd.DataFrame(columns=column_names)

In [ ]:
hls = collections.defaultdict(str)
cts = collections.defaultdict(str)

for i in range(len(df)):
  hls[df['Date'][i]] += df['Headline'][i]
  cts[df['Date'][i]] += df['Contents'][i]

new_df['Date'] = hls.keys()
new_df['Headline'] = hls.values()
new_df['Contents'] = cts.values()
new_df.head()

,Date,Headline,Contents
0,2022-04-11,Data centre S-Reits committed to sustainable g...,THE data centre industry has experienced treme...
1,2022-04-14,"I'm not the kingmaker,' says Sabana Reit direc...",SABANA Industrial Real Estate Investment Trust...
2,2022-04-19,Time to double down on S-Reit investments amid...,"INFLATION is poised to continue rising, as sig..."
3,2022-04-21,Border reopening is godsend to Singapore retai...,"AFTER several stops and starts along the way, ..."
4,2022-04-27,Sustainability-linked loans trump other green ...,PROPERTY firms and real estate investment trus...


In [ ]:
def check_sym(df, i, sym):
  sym_to_find = sym
  if sym_to_find in df['Contents'][i]:
    df[sym][i] = 1


for seit in seit_symbols:
  new_df[seit] = 0

for i in range(len(new_df)):
  for sym in seit_symbols:
    check_sym(new_df, i, sym)

new_df.head()

<ipython-input-48-916ba68d6196>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[sym][i] = 1


,Date,Headline,Contents,AW9U,C2PU,HMN,J85,Q5T,ACV,BMGU,...,O5RU,A17U,DHLU,BWCU,J91U,BUOU,AJBU,ME8U,M44U,M1GU
0,2022-04-11,Data centre S-Reits committed to sustainable g...,THE data centre industry has experienced treme...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2022-04-14,"I'm not the kingmaker,' says Sabana Reit direc...",SABANA Industrial Real Estate Investment Trust...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2022-04-19,Time to double down on S-Reit investments amid...,"INFLATION is poised to continue rising, as sig...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2022-04-21,Border reopening is godsend to Singapore retai...,"AFTER several stops and starts along the way, ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2022-04-27,Sustainability-linked loans trump other green ...,PROPERTY firms and real estate investment trus...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
new_df['Date'] = pd.to_datetime(new_df['Date'])
new_df = new_df.sort_values(by='Date')

# Generate a date range covering the missing dates
date_range = pd.date_range(start=new_df['Date'].min(), end=new_df['Date'].max(), freq='D')
new_df = new_df.set_index('Date').reindex(date_range).reset_index(names=['Date'])

# Fill NaN values in the "Contents" column with the value from the previous row
#new_df['Contents'] = new_df['Contents'].fillna(method='ffill')
new_df.iloc[:, 1:] = new_df.iloc[:, 1:].fillna(method='ffill')

print(new_df.info())

new_df.head(20)

In [ ]:
new_df.iloc[:, 3:] = new_df.iloc[:, 3:].astype(int)

new_df.tail()

In [ ]:
new_df['Headline'] = new_df['Headline'].str.strip('\n\t ')
new_df['Contents'] = new_df['Contents'].str.strip('\n\t ')
new_df.tail()

,Date,Headline,Contents,AW9U,C2PU,HMN,J85,Q5T,ACV,BMGU,...,O5RU,A17U,DHLU,BWCU,J91U,BUOU,AJBU,ME8U,M44U,M1GU
503,2023-08-27,Singapore-based office S-Reits resilient despi...,HEADWINDS for the global office real estate se...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
504,2023-08-28,Brokers’ take: Citi downgrades highly-geared L...,CITI Research downgraded its call on Lendlease...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
505,2023-08-29,Brokers’ take: Citi downgrades highly-geared L...,CITI Research downgraded its call on Lendlease...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
506,2023-08-30,Frasers Centrepoint Trust to divest Changi Cit...,FRASERS : J69U +2.87% Centrepoint Trust : J69U...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
507,2023-08-31,EC World Reit suspends trading in midst of fin...,UNITS of EC World Real Estate Investment Trust...,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
new_df.to_csv('TheBusinessTimes_reindex.csv', index=False)